In [1]:

import mysql.connector
from pymongo import MongoClient
from datetime import datetime, timedelta
import smtplib
from email.mime.text import MIMEText

In [3]:
# MongoDB connection details
MONGO_HOST = 'localhost'
MONGO_PORT = 27017
MONGO_DB = 'BankMDB'
MONGO_COLLECTION = 'user_activities'

# Email configuration
SMTP_SERVER = 'smtp.gmail.com'
SMTP_PORT = 587
EMAIL_USERNAME = 'dhanshreedharpure@gmail.com'
EMAIL_PASSWORD = 'poyf efgx uejw ufnl'
SUPPORT_EMAIL = 'dhanshreedharpure@gmail.com'

In [8]:
def validate_user(username, password):
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        passwd="Dh@ns19hree08",
        database="bankdb"
    )
    cursor = conn.cursor()

    query = "SELECT * FROM `Users` WHERE username = %s AND password = %s"
    cursor.execute(query, (username, password))
    user = cursor.fetchone()

    conn.close()

    return user

In [5]:
def record_activity(username, activity_type, success, ip_address):
    client = MongoClient(MONGO_HOST, MONGO_PORT)
    db = client[MONGO_DB]
    collection = db[MONGO_COLLECTION]

    activity = {
        "username": username,
        "activity_type": activity_type,
        "timestamp": datetime.now(),
        "ip_address": ip_address,
        "success": success
    }

    collection.insert_one(activity)
    client.close()

In [6]:
def send_email(subject, message):
    msg = MIMEText(message)
    msg['Subject'] = subject
    msg['From'] = EMAIL_USERNAME
    msg['To'] = SUPPORT_EMAIL

    server = smtplib.SMTP(SMTP_SERVER, SMTP_PORT)
    server.starttls()
    server.login(EMAIL_USERNAME, EMAIL_PASSWORD)
    server.sendmail(EMAIL_USERNAME, SUPPORT_EMAIL, msg.as_string())
    server.quit()

In [13]:
def main():
    username = input("Enter your username: ")
    password = input("Enter your password: ")

    user = validate_user(username, password)
    if user:
        print("Login successful!")
        record_activity(username, "login_attempt", True, "127.0.0.1")
    else:
        print("Invalid username or password!")
        record_activity(username, "login_attempt", False, "127.0.0.1")

        # Check if account locked
        client = MongoClient(MONGO_HOST, MONGO_PORT)
        db = client[MONGO_DB]
        collection = db[MONGO_COLLECTION]

        recent_failed_attempts = collection.find({
            "username": username,
            "activity_type": "login_attempt",
            "success": False,
            "timestamp": {"$gte": datetime.now() - timedelta(minutes=10)}
        })

        failed_attempt_count = sum(1 for _ in recent_failed_attempts)

        if failed_attempt_count >= 3:
            print("Your account is locked. Please try after some time.")
            send_email("Account Locked", f"Account with username {username} is locked due to multiple failed login attempts.")
        client.close()

if __name__ == "__main__":
    main()

Enter your username:  john.doe
Enter your password:  hashedpassword1


Login successful!
